# Object Detection

In [1]:
!git clone https://github.com/ultralytics/yolov5

!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cv2 as cv
import os
import random
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import albumentations as A
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
import xml.etree.ElementTree as ET
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


In [4]:
# Get train and validation datasets
images_directory = "../dataset/images"
annotations_directory = "../dataset/annotations"

train_split = 0.8

with open("train.txt") as train:
    train_images_filenames_total = train.read().splitlines()

    split_idx = int(train_split * len(train_images_filenames_total))
    train_images_filenames = train_images_filenames_total[:split_idx]
    val_images_filenames = train_images_filenames_total[split_idx:]


with open("test.txt") as test:
    test_images_filenames = test.read().splitlines()

# Filter out images that can not be loaded properly
train_images_filenames = [i for i in train_images_filenames if cv.imread(os.path.join(images_directory, i + ".png")) is not None]
val_images_filenames = [i for i in val_images_filenames if cv.imread(os.path.join(images_directory, i + ".png")) is not None]
test_images_filenames = [i for i in test_images_filenames if cv.imread(os.path.join(images_directory, i + ".png")) is not None]

random.seed(42)
random.shuffle(train_images_filenames)
random.shuffle(val_images_filenames)
random.shuffle(test_images_filenames)

print(len(train_images_filenames), len(val_images_filenames), len(test_images_filenames))

490 123 264


## Convert to YOLO annotations

In [5]:
import shutil
classes = {
    "trafficlight": 0,
    "stop": 1,
    "speedlimit": 2,
    "crosswalk": 3,
}

os.makedirs("../dataset/labels", exist_ok=True)

os.makedirs("dataset/images/train", exist_ok=True)
os.makedirs("dataset/images/val", exist_ok=True)
os.makedirs("dataset/images/test", exist_ok=True)
os.makedirs("dataset/labels/train", exist_ok=True)
os.makedirs("dataset/labels/val", exist_ok=True)
os.makedirs("dataset/labels/test", exist_ok=True)


for img in train_images_filenames:
    shutil.copy(
        f"../dataset/images/{img}.png", f"dataset/images/train/{img}.png")

for img in val_images_filenames:
    shutil.copy(
        f"../dataset/images/{img}.png", f"dataset/images/val/{img}.png")

for img in test_images_filenames:
    shutil.copy(
        f"../dataset/images/{img}.png", f"dataset/images/test/{img}.png")


for path in os.listdir(annotations_directory):
    with open(os.path.join(annotations_directory, path), "r") as xml:
        folder = ""
        if path.strip(".xml") in train_images_filenames:
            folder = "train"
        elif path.strip(".xml") in val_images_filenames:
            folder = "val"
        else:
            folder = "test"


        with open(f"dataset/labels/{folder}/{path.replace('xml', 'txt')}", "w") as txt:
            tree = ET.parse(xml)
            root = tree.getroot()

            size = root.find("size")
            height = int(size.find("height").text)
            width = int(size.find("width").text)

            objects = root.findall("object")

            lines = []
            for object in objects:
                class_index = classes[object.find("name").text]

                xmin = int(object.find("bndbox/xmin").text)
                ymin = int(object.find("bndbox/ymin").text)
                xmax = int(object.find("bndbox/xmax").text)
                ymax = int(object.find("bndbox/ymax").text)

                # middle of bbox
                bbox_x = ((xmax + xmin) / 2) / width
                bbox_y = ((ymax + ymin) / 2) / height
                bbox_width = (xmax - xmin) / width
                bbox_height = (ymax - ymin) / height

                lines.append(
                    f"{class_index} {bbox_x} {bbox_y} {bbox_width} {bbox_height}")

            txt.write("\n".join(lines))


## Dataset

In [6]:
classes = {
    "trafficlight": 0,
    "stop": 1,
    "speedlimit": 2,
    "crosswalk": 3,
}


class TrafficSignDataset(Dataset):
    def __init__(self, annotations_directory, images_filenames, images_directory, transform=None):
        self.annotations_directory = annotations_directory
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image = cv.imread(os.path.join(
            self.images_directory, image_filename + ".png"))
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        image = image / 255.

        boxes, labels = self._get_boxes_and_labels(image_filename)

        image_id = torch.tensor([idx])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id

        if self.transform is not None:
            transformed = self.transform(
                image=image, bboxes=target["boxes"], labels=target["labels"])
            image = transformed["image"]
            target["boxes"] = transformed["bboxes"]
            target["labels"] = transformed["labels"]
            target["boxes"] = torch.as_tensor(target["boxes"], dtype=torch.float32)
            target["labels"] = torch.as_tensor(target["labels"], dtype=torch.int64)

        return image.float(), target

    # https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html
    def _get_boxes_and_labels(self, filename):
        boxes = []
        labels = []
        with open(os.path.join(self.annotations_directory, filename + ".xml")) as xml:
            tree = ET.parse(xml)
            root = tree.getroot()

            objects = root.findall("object")
            for object in objects:
                class_index = classes[object.find("name").text]

                xmin = int(object.find("bndbox/xmin").text)
                ymin = int(object.find("bndbox/ymin").text)
                xmax = int(object.find("bndbox/xmax").text)
                ymax = int(object.find("bndbox/ymax").text)

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(class_index)

        return boxes, labels


In [7]:
num_classes = len(classes)

# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

batch_size = 32
num_workers = 0  # how many processes are used to load the data

train_transform = A.Compose(
    [
        A.Resize(256, 256),
        A.RandomCrop(224, 224),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2,
                           rotate_limit=30, p=0.5),
        ToTensorV2(),
    ],
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=['labels'])
)

val_transform = A.Compose(
    [A.Resize(256, 256), A.CenterCrop(224, 224), ToTensorV2()],
    bbox_params=A.BboxParams(format="pascal_voc", label_fields=['labels'])
)

train = TrafficSignDataset(annotations_directory,
                           train_images_filenames, images_directory, train_transform)
val = TrafficSignDataset(annotations_directory,
                         val_images_filenames, images_directory, val_transform)
test = TrafficSignDataset(annotations_directory,
                          test_images_filenames, images_directory)

train_dataloader = DataLoader(
    train, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True, collate_fn=lambda batch: tuple(zip(*batch)))
val_dataloader = DataLoader(val, batch_size=batch_size,
                            shuffle=False, num_workers=num_workers, drop_last=False, collate_fn=lambda batch: tuple(zip(*batch)))
test_dataloader = DataLoader(
    test, batch_size=1, shuffle=False, num_workers=num_workers, drop_last=False)


Using cuda device


## Two-Stage

In [8]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

# from torchmetrics import JaccardIndex
# metric = JaccardIndex(num_classes=num_classes)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block): Seque

In [9]:
def train_one_epoch(dataloader, model, epoch, optimizer=None, is_train=True):
    if is_train:
        assert optimizer is not None, "When training, please provide an optimizer."

    num_batches = len(dataloader)

    if is_train:
        model.train()
    else:
        model.eval()

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(dataloader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    total_loss = 0.0

    with torch.set_grad_enabled(is_train):
        for images, targets in tqdm(dataloader):

            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            # loss_value = losses_reduced.item()
            # losses_reduced = sum(loss for loss in loss_dict.values())

            if is_train:
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()

                if lr_scheduler is not None:
                    lr_scheduler.step()

            # IMPORTANT: call .item() to obtain the value of the loss WITHOUT the computational graph attached
            total_loss += losses.item()
            # total_jaccard += metric(final_pred.cpu(), target.cpu())

        return total_loss / num_batches # , total_jaccard / num_batches


def evaluate(dataloader, model, metric):
    model.eval()
    
    with torch.set_grad_enabled(False):
        for images, targets in tqdm(dataloader):
            images = list(image.to(device) for image in images)

            outputs = model(images)
            outputs = [{k: v.to("cpu") for k, v in t.items()} for t in outputs]

            # print(outputs)
            res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
            # print(res)

            metric.update(outputs, targets)
        
    return metric

In [10]:
import numpy as np
from tqdm import tqdm
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# torch.cuda.empty_cache()

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

metric = MeanAveragePrecision(box_format="xyxy", iou_type="bboxes", class_metrics=True)

# let's train it for 10 epochs
num_epochs = 10


train_history = {'loss': []}
val_history = {'meanap': []}
best_val_loss = np.inf

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch()
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)



print("Start training...")
for t in range(num_epochs):
  print(f"\nEpoch {t+1}")
  train_loss = train_one_epoch(train_dataloader, model, t, optimizer)
  print(f"Train loss: {train_loss:.3f}")
  # val_loss = train_one_epoch(val_dataloader, model, t, optimizer, is_train=False)
  meanap = evaluate(val_dataloader, model, metric)
  # print(f"Val loss: {val_loss:.3f}")
  print(meanap.compute())

  # save model when val loss improves
  # if val_loss < best_val_loss:
  #   best_val_loss = val_loss
  #   save_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': t}
  #   torch.save(save_dict, 'best_model.pth')

  # # save latest model
  # save_dict = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': t}
  # torch.save(save_dict, 'latest_model.pth')

  # save training history for plotting purposes
  train_history["loss"].append(train_loss)
  # train_history["jaccard"].append(train_jaccard)

  val_history["meanap"].append(meanap)
  # val_history["jaccard"].append(val_jaccard)

Start training...

Epoch 1


100%|██████████| 15/15 [00:27<00:00,  1.83s/it]


Train loss: 0.967


 25%|██▌       | 1/4 [00:00<00:02,  1.13it/s]

[{'boxes': tensor([[1.2084e+02, 5.7886e+00, 1.8824e+02, 6.9866e+01],
        [3.0391e+01, 6.0489e-01, 1.1772e+02, 6.1355e+01],
        [1.2703e+02, 1.4535e+01, 1.9007e+02, 7.1041e+01],
        [1.4467e+02, 1.3775e+01, 1.9795e+02, 6.1324e+01],
        [1.2838e+02, 2.6156e+01, 2.0394e+02, 7.6154e+01],
        [3.0502e+01, 8.0885e-01, 1.1031e+02, 6.4429e+01],
        [1.1922e+02, 1.8288e+01, 1.9274e+02, 7.0183e+01],
        [1.1301e+02, 1.4495e+01, 1.7341e+02, 5.8404e+01],
        [1.1503e+02, 1.9160e+01, 1.6998e+02, 6.3331e+01],
        [4.0930e+01, 0.0000e+00, 1.1595e+02, 6.0292e+01],
        [5.9135e+01, 7.4180e+00, 1.1239e+02, 5.5002e+01],
        [1.5129e+02, 2.0203e+01, 1.8916e+02, 6.2672e+01],
        [1.0087e+02, 6.1126e+00, 1.8362e+02, 5.7246e+01],
        [1.5044e+02, 1.2847e+01, 1.9852e+02, 6.3205e+01],
        [7.3520e+01, 3.7870e-01, 1.3202e+02, 5.9090e+01],
        [1.3516e+02, 0.0000e+00, 1.9467e+02, 5.5940e+01],
        [9.9841e+01, 7.2039e+00, 1.7962e+02, 5.5257e+01],
   

...

[{'boxes': tensor([[ 75.8293,  55.6901, 135.7912, 100.9667],
        [ 53.3596, 173.1680,  67.3543, 187.0435],
        [ 75.9458,  53.5332, 134.6452,  98.3761],
        [ 75.9107,  55.7821, 135.8678, 100.3620],
        [ 53.0001, 173.5527,  68.4938, 186.7958],
        [ 52.0939, 173.3099,  66.3393, 187.5287]]), 'labels': tensor([2, 2, 1, 3, 3, 1]), 'scores': tensor([0.8322, 0.5425, 0.1231, 0.1060, 0.0732, 0.0653])}, {'boxes': tensor([[ 81.5297, 103.5665, 110.6192, 130.2337],
        [ 83.0161,  78.7584, 113.1007, 104.5540],
        [116.7298, 142.0748, 129.2275, 152.5387]]), 'labels': tensor([2, 2, 2]), 'scores': tensor([0.6895, 0.3987, 0.1300])}, {'boxes': tensor([[ 69.3601,  98.4286, 113.8259, 137.3248],
        [ 67.7224,  99.2146, 115.5463, 136.7607],
        [ 67.1983, 100.6716, 114.5547, 136.9014]]), 'labels': tensor([2, 3, 1]), 'scores': tensor([0.7663, 0.0967, 0.0847])}, {'boxes': tensor([[ 41.1358,  58.0124, 136.0508, 164.0060],
        [ 47.4868,  77.5061, 139.4947, 165.9083]

100%|██████████| 4/4 [02:15<00:00, 33.94s/it]

[{'boxes': tensor([[ 55.9073,  50.4160,  96.0456,  86.4144],
        [ 55.6023,  55.0621,  97.3359,  83.3473],
        [ 56.8281,  55.2020,  94.9388,  83.4589],
        [ 51.6270,  43.6359, 103.2116,  90.0712]]), 'labels': tensor([2, 3, 1, 3]), 'scores': tensor([0.6582, 0.1490, 0.0924, 0.0647])}, {'boxes': tensor([[ 39.9795,  39.9993, 123.2418, 109.5949],
        [ 39.8414, 113.3260, 125.5741, 179.3071],
        [ 45.7382, 108.2910, 124.0969, 183.4011],
        [ 41.7740, 122.5984, 123.7413, 187.1884],
        [ 36.0789,  43.8187, 135.4319, 108.6702],
        [ 35.5995,  41.5117, 148.7642, 107.3231],
        [ 33.4778,  89.0732, 132.4067, 207.7948]]), 'labels': tensor([2, 3, 2, 1, 1, 3, 3]), 'scores': tensor([0.7587, 0.3037, 0.2950, 0.2251, 0.1606, 0.0996, 0.0733])}, {'boxes': tensor([[ 90.3332,  56.1623, 106.0255,  70.1477]]), 'labels': tensor([2]), 'scores': tensor([0.7980])}, {'boxes': tensor([[ 70.2082,  61.9220, 152.7478, 134.5577],
        [ 68.0554, 166.6559, 157.3066, 224.0000]

{'map': tensor(0.1419), 'map_50': tensor(0.2802), 'map_75': tensor(0.1096), 'map_small': tensor(0.1270), 'map_medium': tensor(0.2064), 'map_large': tensor(0.2269), 'mar_1': tensor(0.2012), 'mar_10': tensor(0.2863), 'mar_100': tensor(0.2872), 'mar_small': tensor(0.2513), 'mar_medium': tensor(0.3907), 'mar_large': tensor(0.4667), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}

Epoch 4


  0%|          | 0/15 [00:00<?, ?it/s]

## One-Stage

In [17]:
# Might need to be run on linux/WSL
!python3 yolov5/train.py --batch 10 --epochs 30 --data trafficsigns.yaml
# python yolov5/train.py --batch -1 --epochs 3 --data trafficsigns.yaml --workers 0

SyntaxError: invalid syntax (3741266108.py, line 1)

In [ ]:
# Detection
!python3 yolov5/detect.py --weights yolov5/runs/train/exp22/weights/best.pt --img 640 --conf 0.25 --source dataset/images/train/road2.png

In [8]:
# yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', channels=3, classes=4, autoshape=False)
# yolo_model.to(device)

# yolo = {
#     "model": yolo_model,
#     "name": 'yolov5s',
#     "num_epochs": 10,
#     "loss": nn.CrossEntropyLoss(),  # already includes the Softmax activation
#     "optimizer": torch.optim.SGD(yolo_model.parameters(), lr=1e-3)
# }

# print(yolo_model)

    exitcode = _main(fd, parent_sentinel)
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 125, in _main
    prepare(preparation_data)
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 236, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 287, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
  File "c:\Program Files\Python310\lib\runpy.py", line 269, in run_path
    return _run_module_code(code, init_globals, run_name,
  File "c:\Program Files\Python310\lib\runpy.py", line 96, in _run_module_code
    _run_code(code, mod_globals, init_globals,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Ricardo\Desktop\VC2022\Part2\yolov5\train.py", line 26, in <module>
    import torch
  File "C:\Users\Ricardo\AppData\Roaming\Python\Python310\site-packages\torch\__init__.py", line 